In [1]:
import cv2
import numpy as np

# Specify the paths for the 2 files
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"

# Read the network into Memory
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [2]:
# Read image
frame = cv2.imread("single.jpeg")
frameWidth = frame.shape[1]
frameHeight = frame.shape[0]
threshold = 0.1

# Specify the input image dimensions
inWidth = 368
inHeight = 368

# Prepare the frame to be fed to the network
inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)

# Set the prepared object as the input blob of the network
net.setInput(inpBlob)

In [3]:
output = net.forward()

In [4]:
H = output.shape[2]
W = output.shape[3]
# Empty list to store the detected keypoints
points = []
nPoints = 15
frameCopy = np.copy(frame)

for i in range(nPoints):
    # confidence map of corresponding body's part.
    probMap = output[0, i, :, :]

    # Find global maxima of the probMap.
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    
    # Scale the point to fit on the original image
    x = (frameWidth * point[0]) / W
    y = (frameHeight * point[1]) / H

    if prob > threshold : 
        cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

        # Add the point to the list if the probability is greater than the threshold
        points.append((int(x), int(y)))
    else :
        points.append(None)

# cv2.imshow("Output-Keypoints",frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [6]:
Head = 0
Neck = 1
Right_Shoulder = 2
Right_Elbow = 3
Right_Wrist = 4
Left_Shoulder = 5
Left_Elbow = 6
Left_Wrist = 7
Right_Hip = 8
Right_Knee = 9
Right_Ankle =10
Left_Hip =11
Left_Knee =12
Left_Ankle =13
Chest =14
Background = 15

In [5]:
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
for pair in POSE_PAIRS:
    partA = pair[0]
    partB = pair[1]

    if points[partA] and points[partB]:
        cv2.line(frameCopy, points[partA], points[partB], (0, 255, 0), 3)

In [6]:
cv2.imshow('Output-Keypoints', frameCopy)
cv2.imshow('Output-Skeleton', frame)


cv2.imwrite('Output-Keypoints.jpg', frameCopy)
cv2.imwrite('Output-Skeleton.jpg', frame)
# cv2.waitKey(0)

True

In [7]:
print(points)

[(347, 125), (333, 229), (250, 271), (236, 375), (222, 480), (417, 271), (417, 396), (389, 500), (292, 500), (306, 667), (306, 834), (347, 500), (347, 667), (333, 793), (333, 375)]
